# **Modelo predictivo de satisfacción en proyecto solidario**

Datasets: https://drive.google.com/file/d/1pP523QcG0W4gHYq1NbNTZWVjW3xRdHwC/view?usp=sharing

# Etapa 2: Comprensión y Preparación de los datos

In [1]:
# Librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statistics

In [2]:
# Lectura de archivos
d1 = pd.read_excel("Datos/Encuesta+a+Estudiantes+PS_17+de+mayo+de+2023_12.05.xlsx", header=1)
d2 = pd.read_excel("Datos/Estudiantes+PS_INV23+TEC21_17+de+mayo+de+2023_12.22.xlsx", header=1)
d3 = pd.read_excel("Datos/Estudiantes+PS+AD22+TEC21_17+de+mayo+de+2023_12.07.xlsx", header=1)
d4 = pd.read_excel("Datos/Estudiantes+PS+FJ22+TEC21_17+de+mayo+de+2023_12.17.xlsx", header=1)
d5 = pd.read_excel("Datos/Estudiantes+PS+FJ23_17+de+mayo+de+2023_12.28.xlsx", header=1)

## Unión de datos

In [3]:
# Unir datos
raw_data = pd.concat([d1,d2,d3,d4,d5])

# Comprobación de la unión
print(len(d1)+len(d2)+len(d3)+len(d4)+len(d5)==len(raw_data))


True


## Comprensión

In [4]:
# Exploración
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10499 entries, 0 to 368
Data columns (total 43 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                            Non-Null Count  Dtype         
---  ------                                                                                                                                                                                                                                                                                                                                                                                                                            --------------  

In [5]:
raw_data.describe()

,Progreso,Duración (en segundos),Datos de referencia externos,Latitud de ubicación,Longitud de ubicación,CRN,Nombre del destinatario,Correo electrónico del destinatario,Periodo
count,10499.000000,1.049900e+04,0.0,10495.000000,10495.000000,5804.000000,0.0,0.0,1792.000000
mean,99.995428,8.582590e+03,NaN,24.995108,-99.212264,21151.649724,NaN,NaN,10833.835938
std,0.234194,5.214845e+04,NaN,4.066914,10.763758,10630.865378,NaN,NaN,45151.023523
min,88.000000,2.500000e+01,NaN,-37.828000,-123.133700,341.000000,NaN,NaN,2.000000
25%,100.000000,1.190000e+02,NaN,25.644897,-100.325800,23487.000000,NaN,NaN,3.000000
50%,100.000000,1.900000e+02,NaN,25.654600,-100.311000,23886.000000,NaN,NaN,3.000000
75%,100.000000,3.610000e+02,NaN,25.680200,-100.263100,28036.000000,NaN,NaN,3.000000
max,100.000000,1.298823e+06,NaN,59.955000,144.966900,43269.000000,NaN,NaN,202211.000000


In [6]:
# Cantidad de filas y columnas ANTES DE HACER LIMPIEZA
raw_data.shape

(10499, 43)

In [7]:
# Valores de las columnas
"""
for i in raw_data:
    print(i)
    print(raw_data[i].unique())
"""

'\nfor i in raw_data:\n    print(i)\n    print(raw_data[i].unique())\n'

## Limpieza de datos

In [8]:
# Si más del 50 % de la columna está vacía, se considera borrarla dependiendo de los datos que tenga.
col_inutiles = []
for i in raw_data:
    if raw_data[i].isnull().sum()>=len(raw_data)*0.5:
        col_inutiles.append(i)

# Escogemos los índices de las columnas que eliminaremos porque no servirán para el análisis
col_eliminar=[1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 35, 36, 39, 40, 41, 42]

a = 0
for i in col_eliminar:
    col_eliminar[a]=raw_data.columns[i]
    a+=1

In [9]:
# Eliminación de columnas no necesarias
df = raw_data
df = df.drop(labels=col_eliminar, axis=1)


In [10]:
nombres_nuevos = ["Fecha","Progreso","Duración","P1.1","P1.2","P1.3","P1.4","P1.5","P1.6","P1.7","P1.8","P2","P3","P4","P5.1","P5.2","P5.3","P5.4","P6.1","P6.2","NomOSF_Experiencia","OSF","CRN"]

df = df.rename(columns = dict(zip(df.columns, nombres_nuevos)))
df = df.drop(labels=0)# Dataframe unido con columnas renombradas


In [11]:
# Reemplazar valores de 5 y 1 en todas las columnas
for i in df:
    df[i] =df[i].replace({
    "\n5 Totalmente de Acuerdo": 5,
    "\n5 Muy Satisfecho": 5,
    " 4": 4,
    " 3": 3,
    " 2": 2,
    "\n1 Nada Satisfecho": 1,
    "\n1 Nada de Acuerdo": 1
    })

df.head()

,Fecha,Progreso,Duración,P1.1,P1.2,P1.3,P1.4,P1.5,P1.6,P1.7,...,P5.1,P5.2,P5.3,P5.4,P6.1,P6.2,NomOSF_Experiencia,OSF,CRN,Nombre de Experiencia
1,2021-04-22 17:35:09,100,207,4.0,5.0,4.0,5.0,4.0,5.0,5.0,...,4.0,5.0,5.0,5.0,Es complicado conseguir colaboraciones en esto...,Reconocimiento,León Ortigosa ABP,26892.0,NaN,NaN
2,2021-04-22 18:31:39,100,168,4.0,4.0,4.0,4.0,2.0,4.0,4.0,...,4.0,4.0,4.0,4.0,"Pues ojalá que ahora sí me valgan mis horas, n...",Reconocimiento,"Comenzar de Nuevo, A.C.",26918.0,NaN,NaN
3,2021-04-23 04:47:59,100,349,4.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,Lo único que me desagrado del servicio fue la ...,Área de oportunidad,"Casa Paterna La Gran Familia, A.C.",26913.0,NaN,NaN
4,2021-04-23 07:45:14,100,245,5.0,5.0,5.0,3.0,3.0,5.0,5.0,...,5.0,5.0,5.0,5.0,Me gusto mucho la experiencia por la apertura ...,Reconocimiento,"Programa de Acompañamiento de Servicio Social,...",26933.0,NaN,NaN
5,2021-04-23 08:48:17,100,151,4.0,3.0,3.0,3.0,4.0,5.0,5.0,...,4.0,4.0,4.0,4.0,"No sentí que mi trabajo tuviera impacto, fue m...",Área de oportunidad,Casa Monarca Ayuda Humanitaria al Migrante A.B.P.,26931.0,NaN,NaN


## Exploración de datos limpios

In [12]:
# Cantidad de filas y columnas DESPUÉS DE HACER LIMPIEZA
df.shape

(10494, 24)

In [13]:
df.describe()

,Progreso,Duración,P1.1,P1.2,P1.3,P1.4,P1.5,P1.6,P1.7,P1.8,P5.1,P5.2,P5.3,P5.4,OSF
count,10494.000000,1.049400e+04,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,10493.000000,5803.000000
mean,99.995426,8.586621e+03,4.532831,4.588488,4.561422,4.488897,4.304775,4.492328,4.473935,4.460974,4.599638,4.717907,4.723911,4.712761,21150.475788
std,0.234249,5.216055e+04,0.738486,0.694533,0.719511,0.834810,0.945490,0.841104,0.866857,0.839893,0.701904,0.564130,0.576511,0.588220,10631.405239
min,88.000000,2.500000e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,341.000000
25%,100.000000,1.190000e+02,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,5.000000,23487.000000
50%,100.000000,1.900000e+02,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,23886.000000
75%,100.000000,3.617500e+02,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,28036.000000
max,100.000000,1.298823e+06,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,43269.000000


In [14]:
df.dtypes

Fecha                    datetime64[ns]
Progreso                          int64
Duración                          int64
P1.1                            float64
P1.2                            float64
P1.3                            float64
P1.4                            float64
P1.5                            float64
P1.6                            float64
P1.7                            float64
P1.8                            float64
P2                               object
P3                               object
P4                               object
P5.1                            float64
P5.2                            float64
P5.3                            float64
P5.4                            float64
P6.1                             object
P6.2                             object
NomOSF_Experiencia               object
OSF                             float64
CRN                              object
Nombre de Experiencia            object
dtype: object

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10494 entries, 1 to 368
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  10494 non-null  datetime64[ns]
 1   Progreso               10494 non-null  int64         
 2   Duración               10494 non-null  int64         
 3   P1.1                   10493 non-null  float64       
 4   P1.2                   10493 non-null  float64       
 5   P1.3                   10493 non-null  float64       
 6   P1.4                   10493 non-null  float64       
 7   P1.5                   10493 non-null  float64       
 8   P1.6                   10493 non-null  float64       
 9   P1.7                   10493 non-null  float64       
 10  P1.8                   10493 non-null  float64       
 11  P2                     10493 non-null  object        
 12  P3                     10493 non-null  object        
 13  P4 

In [16]:
no_terminados=df[df["Progreso"]<100]
no_terminados

,Fecha,Progreso,Duración,P1.1,P1.2,P1.3,P1.4,P1.5,P1.6,P1.7,...,P5.1,P5.2,P5.3,P5.4,P6.1,P6.2,NomOSF_Experiencia,OSF,CRN,Nombre de Experiencia
2000,2021-07-22 17:15:42,88,58,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,Ñ,NaN,Fundacionplan.org | Plan SPAP A.C.,346.0,NaN,NaN
3251,2021-11-23 08:19:20,88,6989,4.0,4.0,5.0,5.0,5.0,4.0,5.0,...,5.0,5.0,5.0,5.0,El propósito es noble y me gusta formar parte ...,NaN,Dirección de Servicio Social - Aprendizajes pa...,23478.0,NaN,NaN
4434,2021-12-02 13:27:21,88,179,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,Me gusto mucho ser parte de este proyecto soli...,NaN,Centro Virtual de Aprendizaje Tec de Monterrey,23492.0,NaN,NaN
1340,2022-06-09 08:39:33,88,68,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,Me gustó la actitud de los muchachos,NaN,Dirección de Servicio Social - Aprendizajes pa...,28008.0,NaN,Aprendamos Juntos: Coordinador Académico Matem...
